Test de bondad de ajuste al borde real

In [1]:
def test_BR(bordes, bordes_ref, rango=0.0075):
    """Calcula precision y recall conforme a lo que se hace habitualmente en superpixeles, datos en binario"""
    import numpy as np
    acierto=error=acierto2=error2=0
    A=0
    total=np.sum(bordes_ref)
    total2=np.sum(bordes)
    dim1=bordes_ref.shape[0]
    dim2=bordes_ref.shape[1]
    diag=int(np.sqrt(dim1**2 + dim2**2))
#     print(dim1**2,dim2**2)
    distancia=int(diag*rango)
#     print("diagonal de la imagen: ",diag,"(",bordes_ref.shape[0],"x",bordes_ref.shape[1],")")
#     print("distancia: ",distancia)
    
    for i in range(dim1):
        for j in range(dim2):
            if bordes_ref[i][j]==1:                
                mascara=np.zeros((dim1,dim2),np.uint8)
                mascara[i-distancia:i+distancia,j-distancia:j+distancia]=1
                resultado=bordes*mascara
                resultado=np.sum(resultado)
                if resultado>=1:acierto+=1
                else: error+=1
            if bordes[i][j]==1:
                mascara2=np.zeros((dim1,dim2),np.uint8)
                mascara2[i-distancia:i+distancia,j-distancia:j+distancia]=1
                resultado2=bordes_ref*mascara2
                resultado2=np.sum(resultado2)
                if resultado2>=1:acierto2+=1
                else: error2+=1
    
#     print("he recorrido: ",i,"-",j)
#     print("aciertos:",acierto," - errores:", error, "total: ",total)
#     print("aciertos2:",acierto2," - errores2:", error2, "total2: ",total2)
    
    if (acierto+error)!=0: recall=acierto/(acierto+error)
    else : recall=0
    print("recall:",round(recall,4))
    if (acierto+error2)!=0: precision=acierto/(acierto+error2)
    else : precision=0
    print("precision:",round(precision,4))
    
    return (round(recall,4),round(precision,4))
    
                
    
    
    
    

In [2]:
def ASA(segmentacion, segmentacion_ref):
    """Dadas 2 segmentaciones calcula el parecido, asignando a la evaluada en cada segmento el valor mas repetido
    en el mismo area de la de referencia"""
    
    import numpy as np
    import cv2
    
    if 0 in np.unique(segmentacion): segmentacion=segmentacion+1
    if 0 in np.unique(segmentacion_ref): segmentacion_ref=segmentacion_ref+1
    
    aux=np.copy(segmentacion)
    res=np.copy(segmentacion)
    matrizASA=np.copy(segmentacion)
    for seg in np.unique(segmentacion):
        mask=np.where(segmentacion==seg,1,0)
        tamano=np.sum(mask)
        aux=mask*segmentacion_ref
        valor,frec=np.unique(aux, return_counts=True)
#         print("valores en segmento:", valor, "frecuencias:",frec)
        valor_rep=valor[np.argmax(frec[1:])+1]
#         print(np.argmax(frec[1:]))
#         print("Segmento:",seg, "valor en referencia:", valor_rep, "aparece: ",frec,"veces de:",tamano)
        res=np.where(segmentacion==seg,valor_rep,res)
    matrizASA=np.where(res==segmentacion_ref,1,0)
    ASA=np.sum(matrizASA)/(matrizASA.shape[0]*matrizASA.shape[1])
    print("aciertos: ",np.sum(matrizASA), "total: ",matrizASA.shape[0]*matrizASA.shape[1], "ASA:",round(ASA,4))
    
    return round(ASA,4)